# Politische Reden als Datenquelle

In [ ]:
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
import seaborn as sns
from functools import reduce
import numpy as np
from pandarallel import pandarallel
import re

In [ ]:
load_dotenv()
sns.set(style="white", palette="muted", rc={"figure.figsize": (20, 8)})
pandarallel.initialize()

In [ ]:
FAST_MODE = False
data_dir = Path("../../data/speeches")

## Business Understanding

Im Folgenden wird der [Datensatz bestehend aus Bundestagsreden](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/FIKIBO), bereitgestellt von [Open Discourse](https://opendiscourse.de/) initial untersucht. Politische Reden, in diesem Fall aus dem deutschen Bundestag, sollen als ein Standbein als Trainingsdaten für das Klassifikationsproblem genutzt werden. Der Datensatz bietet den Vorteil, dass eine große Menge an gelabelten Texten zur Verfügung steht sowie alle möglichen Themenbereiche abdeckt, die im Laufe der Jahre im Bundestag debattiert wurden. Da die Reden deutlich länger sind als Tweets (siehe unten), muss eine Lösung gefunden werden, wie die Redetexte aufgeteilt werden können, um sie dem Modell als Input zur Verfügung zu stellen. Andererseits kann dies auche als Vorteil gesehen werden, da die Reden deutlich mehr Informationen enthalten als Tweets und somit dem Modell eine bessere Chance bieten, auch aufgrund inhaltlichen Zusammenhängen zu entscheiden.

## Data Understanding

### `factions`


In [ ]:
df_factions = pd.read_feather(data_dir / "src" / "factions.feather")

In [ ]:
df_factions

Die `factions`-Datei verknüpft die ID der Parteien mit deren Namen. Für die für uns relevanten Parteien sollten wir uns frühzeitig auf ein einheitliches Mapping zwischen Label und Partei einigen, um spätere Probleme zu vermeiden.


### `politicians`


In [ ]:
df_politicians = pd.read_feather(data_dir / "src" / "politicians.feather")

In [ ]:
df_politicians

Die `politicians`-Datei verknüpft die ID der Politiker mit deren Namen inkl. weiteren Informationen. Da wir zunächst nur nach Parteien klassifizieren wollen, ist diese Datei für uns nicht relevant.


In [ ]:
sns.countplot(x="faction", data=df_politicians)

### `electoral_terms`


In [ ]:
df_electoral_terms = pd.read_feather(data_dir / "src" / "electoral_terms.feather")

In [ ]:
df_electoral_terms

Die `electoral_terms`-Datei verknüpft die ID der Wahlperioden mit derem Start- und Enddatum. Um in allen Datenquellen vergleichbare Themen, Sprache und Parteien zu haben, sollten alle Daten auf einen ähnlichen Zeitraum beschränkt werden. Da sich die uns zur Verfügung stehenden Tweet Daten auf die Bundestagswahl 2021 und die Wahlperiode 19 beziehen, sind vor allem Reden als der 19. Wahlperiode relevant.


### `contributions`


In [ ]:
df_contributions = pd.read_feather(data_dir / "src" / "contributions_extended.feather")

### `speeches`


In [ ]:
df_speeches = pd.read_feather(data_dir / "src" / "speeches.feather", columns=["electoralTerm", "speechContent", "politicianId", "factionId"])

In [ ]:
df_speeches

Wie gerade angemerkt, sind für uns erstmal die Reden aus der 19. Wahlperiode relevant:


In [ ]:
df_speeches = df_speeches[df_speeches["electoralTerm"] == 19].reset_index()
df_speeches

Aus dieser Wahlperiode bleiben zunächst etwa 61.000 Reden übrig. Davon sind jedoch noch jene abzuziehen, die keiner Partei zugeordnet sind, weil beispielsweise das Präsidium des Bundestage gesprochen hat. Zudem wird mithilfe der `factionId` der Namen der Fraktion bestimmt.


In [ ]:
factions_of_interest = [0, 3, 4, 6, 13, 23]
df_speeches = df_speeches[df_speeches["factionId"].isin(factions_of_interest)]

party_mapping = {
    0: "AfD",
    3: "Grüne",
    4: "Union",
    6: "Linke",
    13: "FDP",
    23: "SPD"
}
df_speeches["faction"] = df_speeches["factionId"].map(party_mapping)
df_speeches = df_speeches.drop(columns=["factionId"])

df_speeches

Somit reduziert sich die Anzahl an verbleibender Reden auf knapp 29.000.


Da der Wert von `electoralTerm` nun konstant ist, kann die Spalte gelöscht werden:

In [ ]:
df_speeches = df_speeches.drop(columns=["electoralTerm"])
df_speeches

#### `Anzahl an Reden pro Fraktion`

Wenn man sich die Anzahl an Reden pro Fraktion anschaut, fällt auf, dass die Fraktionen der CDU/CSU und der SPD deutlich mehr Reden haben als die anderen Parteien, was auch zu erwarten ist. Im Zuge der Data Preparation muss hierzu sichergestellt werden, dass ein Ungleichgewicht vermieden wird.


In [ ]:
sns.countplot(x="faction", data=df_speeches)

In [ ]:
df_len = df_speeches["speechContent"].str.len()
df_len.describe()

In [ ]:
sns.histplot(df_len)

Auffallend sie viele Reden mit keinem oder sehr wenigen Zeichen. Damit eine Rede auch inhaltlich verarbeitet werden kann, muss im Zuge der Data Preparation eine Mindestlänge festgelegt werden.


In [ ]:
df_speeches = df_speeches[df_speeches["speechContent"].str.len() > 400]
df_speeches = df_speeches[df_speeches["speechContent"].str.len() < 7000]
df_speeches

Setzt man beispielsweise voraus, dass jede Rede mindestens 100 Zeichen umfassen muss, so bleiben immer noch etwa 26.500 Reden übrig.

Es fällt zudem auf, dass die meisten Reden mit einer Anrede an den Bundestagspräsidenten und das Plenum beginnen sowie mit einem Dank zum Schluss aufhören. Hier ist fraglich, inwieweit diese Anreden und Danksagungen relevant sind. Im Zuge der Data Preparation muss hierzu eine Entscheidung getroffen werden und gegebenenfalls versucht werden, diese Passagen zu entfernen, damit die Reden nur die inhaltlichen Ausführungen enthalten.


### Test if maximum number of tokens (512 for BERT based models) is exceeded

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "oliverguhr/german-sentiment-bert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
df_speeches["num_tokens"] = df_speeches["speechContent"].apply(lambda x: len(tokenizer.encode(x)))

In [ ]:
df_speeches["num_tokens"].describe()

In [ ]:
reduce(lambda a, b: set((*a, *b)), df_speeches["speechContent"].apply(np.array))

Schaut man sich an, welche Zeichen in den Reden vorkommen, fällt auf, dass neben dem regulären Alphabet auch einige unerwünschte Sonderzeichen enthalten sind, die im Zuge der Data Preparation entfernt werden müssen.


## Data Preparation

In [ ]:
from studienarbeit.utils.cleaning import CleanText

In [ ]:
df_prep = pd.read_feather(data_dir / "src" / "speeches.feather", columns=["electoralTerm", "speechContent", "factionId"])

In [ ]:
df_prep.head()

### Encoding

In [ ]:
party_mapping = {
    0: 0,   # AfD
    3: 2,   # Grüne
    4: 5,   # Union
    6: 3,   # Linke
    13: 1,  # FDP
    23: 4,  # SPD
}

In [ ]:
df_prep["party"] = df_prep["factionId"].map(party_mapping)

In [ ]:
df_prep

In [ ]:
def initial_cleaning(text):
    text = re.sub("[\u2022\u2023\u25E6\u2043\u2219\uf0b7\u25fc]\s", "", text)
    text = re.sub("({\d*})", "", text)
    text = re.sub("\(\w*\)", "", text)
    text = text.replace(". –", ".")
    
    text = re.sub("\n", " ", text)
    text = re.sub("\t", " ", text)
    text = re.sub("\s+", " ", text)
    text = text.strip()
    
    return text

In [ ]:
df_prep["init_clean"] = df_prep["speechContent"].parallel_apply(lambda x: initial_cleaning(x))
df_prep = df_prep.drop(columns=["speechContent", "factionId"])

In [ ]:
if FAST_MODE and (data_dir / "cache/speeches_prep.feather").exists():
    df_prep = pd.read_feather(data_dir / "cache/speeches_prep.feather")
else:
    clean = CleanText()
  
    df_prep["clean_text"] = df_prep["init_clean"].parallel_apply(lambda x: clean.clean_text(x, True))
    df_prep["tokenized_text"] = df_prep["clean_text"].parallel_apply(lambda x: clean.remove_stopwords(clean.stemm_text(x)))
    
    if (data_dir / "cache").exists() == False:
        (data_dir / "cache").mkdir()
    df_prep.to_feather(data_dir / "cache/speeches_prep.feather")

In [ ]:
df_prep.to_csv(data_dir / "cache/speeches_prep.csv", index=False)

In [ ]:
df_prep

In [ ]:
df_prep["num_tokens"] = df_prep["clean_text"].apply(lambda x: len(tokenizer.encode(x)))

In [ ]:
df_prep

In [ ]:
sns.histplot(df_prep["num_tokens"])

## Modelling

In [ ]:
df_final = df_prep.copy().reset_index(drop=True).drop(columns=["init_clean"])

In [ ]:
df_final

In [ ]:
df_final.to_parquet(data_dir / "speechess.parquet", index=False)